In [2]:
from hap import HomeAutoPilot
from hap_util import *

# data prep

In [3]:
areas =  ['backyard', 'frontyard', 'garage', 'kitchen', 'livingroom']
data_files = [f'../data/{area}.csv' for area in areas]
input_tensors, act_dev_tensors, act_state_tensors, act_timing_tensors, n_features, n_act_state, n_act_dev, unique_entity_ids, unique_event_states = prep_tensors(data_files)

100% 15453/15453 [00:01<00:00, 8197.15it/s]

number of unique actionalble devices: 94, number of actionable state: 2
number of features: 9



/root/workspaces/hap/hap_util.py:105: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  input_tensors = [torch.tensor(input_sequence).to(torch.float32) for input_sequence in input_sequences]


In [4]:
input_tensors = torch.load('input_seq_tensors.pt', weights_only=True)
act_dev_tensors = torch.load('act_dev_tensors.pt', weights_only=True)
act_state_tensors = torch.load('act_state_tensors.pt', weights_only=True)
act_timing_tensors = torch.load('act_timing_tensors.pt', weights_only=True)
device_id_map = torch.load('device_id_map.pt', weights_only=False)
device_state_id_map = torch.load('device_state_map.pt', weights_only=False)

n_act_state = len(device_state_id_map)
n_act_dev = len(device_id_map)

# run inferencing

In [57]:
# re-create the model
pilot = HomeAutoPilot(num_devices = n_act_dev, num_device_actions = n_act_state)
pilot.load_model('pilot_demo2.ptmodel')

INFO:hap:Using device: cuda


In [161]:
import random

sample_idx = random.choice(range(len(input_tensors)))
input_tensor = input_tensors[sample_idx]
predicted_timing, predicted_device_idx, predicted_action_idx = pilot.eval(input_tensor)
device_name = device_id_map[predicted_device_idx]
action_name = device_state_id_map[predicted_action_idx]

print(f'prediction: Turn {device_name} {action_name} after {predicted_timing} seconds')
print('----ground truth----')
gt_dev_name = device_id_map[act_dev_tensors[sample_idx].int().item()]
gt_action_name = device_state_id_map[act_state_tensors[sample_idx].int().item()]
gt_timing = act_timing_tensors[sample_idx].item()
print(f'ground turth: Turn {gt_dev_name} {gt_action_name} after {gt_timing} seconds')

prediction: Turn switch.garage_light on after 8.813950538635254 seconds
----ground truth----
ground turth: Turn light.garage_lamp off after 0.0 seconds


# train model

In [56]:
pilot = HomeAutoPilot(num_devices = n_act_dev, num_device_actions = n_act_state)
pilot.train_model(input_tensors, act_dev_tensors, act_state_tensors, act_timing_tensors, train_epoch = 10, model_path = 'pilot_demo2.ptmodel')

INFO:hap:Using device: cuda
3it [00:00, 92.17it/s]
INFO:hap:Epoch [1/10], Loss: 1435.6392
3it [00:00, 93.40it/s]
INFO:hap:Epoch [2/10], Loss: 2411.0146
3it [00:00, 114.82it/s]
INFO:hap:Epoch [3/10], Loss: 435.6131
3it [00:00, 96.89it/s]
INFO:hap:Epoch [4/10], Loss: 1375.7239
3it [00:00, 92.91it/s]
INFO:hap:Epoch [5/10], Loss: 1487.9615
3it [00:00, 99.71it/s]
INFO:hap:Epoch [6/10], Loss: 5067.0942
3it [00:00, 82.89it/s]
INFO:hap:Epoch [7/10], Loss: 259.8890
3it [00:00, 104.70it/s]
INFO:hap:Epoch [8/10], Loss: 9946.4688
3it [00:00, 97.10it/s]
INFO:hap:Epoch [9/10], Loss: 1007.7250
3it [00:00, 105.69it/s]
INFO:hap:Epoch [10/10], Loss: 8514.4121
INFO:hap:Test Loss: 4217.8364
INFO:hap:Device Prediction Accuracy: 0.6250
INFO:hap:Action Prediction Accuracy: 0.4375
